# Agentic AI Tool Use Demo with LangChain and Pydantic

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
gemini_model = "gemini-2.5-flash"

## 1. Defining Tools using Pydantic Schemas

In [ ]:
from langchain.tools import tool
from pydantic import BaseModel, Field

In [ ]:
# Schema Definitions
class WeatherInput(BaseModel):
    city: str = Field(description="The city to get weather for")
    unit: str = Field(default="celsius", description="Temperature unit: celsius or fahrenheit")

class CalculatorInput(BaseModel):
    expression: str = Field(description="Math expression to evaluate, e.g. '2 + 2'")

class SearchInput(BaseModel):
    query: str = Field(description="Search query string")
    max_results: int = Field(default=3, description="Maximum number of results to return")

### Tool Definitions - LLM sees the docstrings

In [ ]:
@tool(args_schema=WeatherInput)
def get_weather(city: str, unit: str = "celsius") -> str:
    """Get the current weather for a given city."""
    # Simulated response for demo purposes
    return f"Weather in {city}: 22°{unit[0].upper()}, partly cloudy."

@tool(args_schema=CalculatorInput)
def calculator(expression: str) -> str:
    """Evaluate a simple math expression."""
    try:
        result = eval(expression, {"__builtins__": {}})  # restricted eval
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {e}"

@tool(args_schema=SearchInput)
def web_search(query: str, max_results: int = 3) -> str:
    """Search the web for information."""
    # Simulated response for demo purposes
    results = [f"Result {i+1} for '{query}'" for i in range(max_results)]
    return "\n".join(results)

In [ ]:
tools = [get_weather, calculator, web_search]
print("Tools registered:", [t.name for t in tools])

In [ ]:
# Looking at the tool schemas
import json

for t in tools:
    print(f"\nTool: {t.name}")
    print(f"Description: {t.description}")
    print(f"Schema: {json.dumps(t.args_schema.model_json_schema(), indent=2)}")

### Manually Invoking the Tools

In [ ]:
print(get_weather.invoke({"city": "Paris", "unit": "celsius"}))
print(calculator.invoke({"expression": "15 * 4 + 3"}))
print(web_search.invoke({"query": "Agentic AI", "max_results": 2}))

## 2. Binding Tools to Gemini

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model=gemini_model, temperature=0)
llm_with_tools = llm.bind_tools(tools)

print("LLM with tools bound:", llm_with_tools)

## 3. **Full Loop:** user message -> tool decision -> tool call -> final response from LLM 

### Use the LLM to decide which tool to call

In [ ]:
from langchain_core.messages import HumanMessage

user_message = HumanMessage(content="What is the weather in Philadelphia, Pennsylvania?")

response = llm_with_tools.invoke([user_message])

print("LLM Response:")
print("Content:", response.content)
print("Tool Calls:", response.tool_calls)

### Execute the Tool Call

In [ ]:
from langchain_core.messages import ToolMessage

# Map tool names to tool objects
tool_map = {t.name: t for t in tools}

# Execute each tool the LLM requested
tool_messages = []
for tool_call in response.tool_calls:
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]
    tool_result = tool_map[tool_name].invoke(tool_args)
    
    print(f"Executed tool: {tool_name}({tool_args}) -> {tool_result}")
    
    tool_messages.append(
        ToolMessage(content=tool_result, tool_call_id=tool_call["id"])
    )

### Complete the loop by sending the tool results back to the LLM

In [ ]:
final_response = llm_with_tools.invoke(
    [user_message] + tool_messages
)

print("Final LLM Response:")
print("Content:", final_response.content)